In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.styles.stylesheet")


## Preprocess Eurostat Dataset
The Eurostat datasets have a unique format for which we need to write a pre-processing function: Datasets are stored in Excel files consisting of a "Summary Sheet" describing the Data Sheets and their "feature configuration". The following function aims at simplifying the pre-processing of these excel files.

In [2]:
def process_eurostat_dataset(path: str):

    # retrieve sheets data and metadata from Summary sheet
    ## the time frequency and unit of measurement information 
    # is not a relevant index value since they are always the same
    ignore = ['Time frequency', 'Unit of measure']
    summary = pd.read_excel(
        path, sheet_name='Summary', 
        usecols=lambda col: \
            not col.startswith('Unnamed') and col not in ignore,
        index_col='Contents', skiprows=14
    )

    all = []

    # iterate over data sheets and pre-process them
    for sheet_name, config in summary.iterrows():

        # the valuable information starts with the row containing "TIME" indicating the start of the data table
        preview = pd.read_excel(path, sheet_name=sheet_name, usecols=[0])
        header_row = preview[preview.iloc[:, 0] == 'TIME'].index[0]

        data = pd.read_excel(
            path, sheet_name=sheet_name,
            usecols= lambda col: not str(col).startswith('Unnamed'),
            na_values=':', skiprows=header_row + 1
        )

        # Remove the first row "Geo (Label)"
        data = data.iloc[1:].reset_index(drop=True)

        # some sheets contain additional meta-data at the end which are seperated by a NaN value row from the actual data
        ## if we find such a row we remove all rows after
        first_blank_row_index = data.isnull().all(axis=1).idxmax() if data.isnull().all(axis=1).any() else None
        if first_blank_row_index is not None:
            data = data.loc[:first_blank_row_index - 1]

        # create multi-index based on config
        config_df = pd.DataFrame([config] * len(data), index=data.index)
        data = pd.concat([config_df, data], axis=1)
        data.set_index(['TIME'] + list(config_df.columns), inplace=True)

        all.append(data)

    # merge them into one multi-index data frame 
    all = pd.concat(all).sort_index()
    
    return all

In [3]:
FILE_PATH = 'data/nrg_inf_epcrw_spreadsheet.xlsx'
data = process_eurostat_dataset(FILE_PATH)

In [4]:
data

1990  \
TIME           Standard international energy product classific... Technical characteristics of plants                       
Albania        Biogases                                           Net maximum electrical capacity                       0   
                                                                  Net maximum electrical capacity - alternating c...  NaN   
                                                                  Net maximum electrical capacity - direct current    NaN   
               Geothermal                                         Net maximum electrical capacity                       0   
                                                                  Net maximum electrical capacity - alternating c...  NaN   
...                                                                                                                   ...   
United Kingdom Wind off shore                                     Net maximum electrical capacity - alternating c...  NaN   
                                                                  Net maximum electrical capacity - direct current    NaN   
               Wind on shore                                      Net maximum electrical capacity                       0   
                                                                  Net maximum electrical capacity - alternating c...  NaN   
                                                                  Net maximum electrical capacity - direct current    NaN   

                                                                                                                      1991  \
TIME           Standard international energy product classific... Technical characteristics of plants                        
Albania        Biogases                                           Net maximum electrical capacity                      0.0   
                                                                  Net maximum electrical capacity - alternating c...   NaN   
                                                                  Net maximum electrical capacity - direct current     NaN   
               Geothermal                                         Net maximum electrical capacity                      0.0   
                                                                  Net maximum electrical capacity - alternating c...   NaN   
...                                                                                                                    ...   
United Kingdom Wind off shore                                     Net maximum electrical capacity - alternating c...   NaN   
                                                                  Net maximum electrical capacity - direct current     NaN   
               Wind on shore                                      Net maximum electrical capacity                      0.0   
                                                                  Net maximum electrical capacity - alternating c...   NaN   
                                                                  Net maximum electrical capacity - direct current     NaN   

                                                                                                                      1992  \
TIME           Standard international energy product classific... Technical characteristics of plants                        
Albania        Biogases                                           Net maximum electrical capacity                      0.0   
                                                                  Net maximum electrical capacity - alternating c...   NaN   
                                                                  Net maximum electrical capacity - direct current     NaN   
               Geothermal                                         Net maximum electrical capacity                      0.0   
                                                                  Net maximum electrical capacit

In [7]:
print(data.index.levels)

[['Albania', 'Austria', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Euro area – 20 countries (from 2023)', 'European Union - 27 countries (from 2020)', 'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kosovo*', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Türkiye', 'Ukraine', 'United Kingdom'], ['Biogases', 'Geothermal', 'Hydro', 'Industrial waste (non-renewable)', 'Mixed hydro power', 'Municipal waste', 'Other liquid biofuels', 'Pumped hydro power', 'Pure biodiesels', 'Pure biogasoline', 'Pure hydro power', 'Run-of-river hydro power', 'Solar', 'Solar photovoltaic', 'Solar photovoltaic (1+ MW)', 'Solar photovoltaic (1+ MW, off grid)', 'Solar photovoltaic (1+ MW, roof top)', 'Solar photovoltaic (20 kW - 1000 kW

In [6]:
print(len(data.index.levels[0]))

43
